# Setup

In [1]:
import sys
import os

import re
import collections
import itertools
import bcolz
import pickle

import numpy as np
import pandas as pd
import gc
import random
import smart_open
import h5py
import csv

import tensorflow as tf
import gensim
import string

import datetime as dt
from tqdm import tqdm_notebook as tqdm

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

random_state_number = 967898

# Code

## Day 1: Inverse Captcha

In [2]:
! cat day1_input.txt

3181317434923597215981186975516634388295837647427843768163249522249921148864954375565513884255386724613124546288175686273692292575264734167334275651485666397949674715824179285762547132353518322249794975164448827731717349612447389345242511813364598448875912889714649883137379572166169649262227628288121837127397353816377978243521149119661637513547251793548196443995684453613682375776449496729725154538946447279447444794156477873392653274175275786524394697626642654834188987351438346414265942512278666739914333577217497312838386989332597731965183951669429553414666872882239345262632189235719257444485626472158536516494564725464526469395789837321489784842496626658299127249677115958371545671464558557664145835832652185851831931523385747369571223832378725455659756646118845227985376618433369634439581861521584634858654116419462437135355681254894544743278779548944331294168722131443269411584786312982653262822838689468339235279951494266539627372682193634666348549915914136844378247571467995321338837593951

In [18]:
input_data = None
with open("day1_input.txt") as f:
    input_data = f.read().strip().split()[0]

### part1

The captcha requires you to review a sequence of digits (your puzzle input) and 

find the sum of all digits that match the next digit in the list. The list is circular, 

so the digit after the last digit is the first digit in the list.

In [68]:
def get_captcha11(s):
    ssum = 0
    for i1, i2 in zip(s, s[1:]+s[0]):
        if i1 == i2:
            ssum += int(i1)
    return ssum

In [69]:
def get_captcha12(s):
    n = len(s)
    ssum = int(s[0]) if s[0] == s[-1] else 0
    i1 = 0
    while i1 < n-1:
        if s[i1] == s[i1+1]:
            ssum += int(s[i1])
        i1+=1
    return ssum

In [70]:
assert  get_captcha12("1122") == 3
assert  get_captcha12("1111") == 4
assert  get_captcha12("1234") == 0
assert  get_captcha12("91212129") == 9

In [71]:
get_captcha12(input_data)

1203

### part2

Now, instead of considering the next digit, it wants you to consider the digit halfway around 

the circular list. That is, if your list contains 10 items, only include a digit in your sum 

if the digit 10/2 = 5 steps forward matches it. Fortunately, your list has an even number of elements.

In [88]:
def get_captcha21(s):
    n = len(s)
    ssum = 0
    i1 = 0
    halfway_circle = n//2
    while i1 < n:
        if s[i1] == s[(i1+halfway_circle)%n]:
            ssum += int(s[i1])
        i1+=1
    return ssum

In [89]:
assert  get_captcha21("1212") == 6
assert  get_captcha21("1221") == 0
assert  get_captcha21("123425") == 4
assert  get_captcha21("123123") == 12
assert  get_captcha21("12131415") == 4

In [90]:
get_captcha21(input_data)

1146

## Day 2: Corruption Checksum

In [92]:
! cat day2_input.txt

116	1259	1045	679	1334	157	277	1217	218	641	1089	136	247	1195	239	834
269	1751	732	3016	260	6440	5773	4677	306	230	6928	7182	231	2942	2738	3617
644	128	89	361	530	97	35	604	535	297	599	121	567	106	114	480
105	408	120	363	430	102	137	283	123	258	19	101	181	477	463	279
873	116	840	105	285	238	540	22	117	125	699	953	920	106	113	259
3695	161	186	2188	3611	2802	157	2154	3394	145	2725	1327	3741	2493	3607	4041
140	1401	110	119	112	1586	125	937	1469	1015	879	1798	122	1151	100	926
2401	191	219	607	267	2362	932	2283	889	2567	2171	2409	1078	2247	2441	245
928	1142	957	1155	922	1039	452	285	467	305	506	221	281	59	667	232
3882	1698	170	5796	2557	173	1228	4630	174	3508	5629	4395	180	5100	2814	2247
396	311	223	227	340	313	355	469	229	162	107	76	363	132	453	161
627	1331	1143	1572	966	388	198	2068	201	239	176	1805	1506	1890	1980	1887
3390	5336	1730	4072	5342	216	3823	85	5408	5774	247	5308	232	256	5214	787
176	1694	1787	1586	3798	4243	157	4224	3603	2121	3733	851	2493	4136	148	153
2432	4030	3397	4032	3952

In [118]:
input_data = []
with open("day2_input.txt") as f:
    for line in f.read().split("\n"):
        input_data += [list(map(int,line.split("\t")))] if line else []

### part 1

For each row, determine the difference between the largest value and the 

smallest value; the checksum is the sum of all of these differences.

In [124]:
def get_checksum1(ll):
    return sum([max(l) - min(l) for l in ll])

In [125]:
assert get_checksum1([[5,1,9,5],[7,5,3],[2,4,6,8]]) == 18

In [126]:
 get_checksum1(input_data)

41887

### part 2

the goal is to find the only two numbers in each row where one evenly 

divides the other - that is, where the result of the division operation is 

a whole number. They would like you to find those numbers on each line, 

divide them, and add up each line's result.

In [137]:
def get_checksum2(ll):
    csum = 0
    for l in ll:
        ht = {}
        for i,a in enumerate(l):
            for b in l[i:]:
                if a == b:
                    continue
                if a%b == 0:
                    csum += a//b
                    break
                elif b%a == 0:
                    csum += b//a
                    break
    return csum

In [138]:
assert get_checksum2([[5,9,2,8],[9,4,7,3],[3,8,6,5]]) == 9

In [139]:
 get_checksum2(input_data)

226

## Day 3: Spiral Memory

### part 1

In [247]:
import math

def get_md_spiral_mem(n):

    # this number marks the odd square end from where the counting starts.
    # this counting is used to estimate the catesian cor ordinate which
    # can be used to find the manhattan distance
    ref = math.floor(math.sqrt(n))
    
    # the spiral starts after the squared odd number
    # if this is even reduce by 1 to get this place for reference
    ref = ref - 1 if ref%2==0 else ref
    
    # the number can be on either of the 4 sides when spiraling
    # find difference from the squared odd number reference
    diff = n - ref**2
    
    # since all are odd number squares
    # and the side considers the full side including 
    # the number that starts on the next side
    side_size = ref+1
    print("------", ref, diff, side_size)
    # lets estimate the absolutre value of thier
    # cartesian co ordinates. Since the reference is taken from the odd
    # square numbers, handle them properly, this part is not implemented
    # also this can be done pretty simply using comprehension,
    # but this way the code is understandable
    
    x,y = None, None
    if n <= ref**2 + side_size:
        # right side
        x = side_size//2
        d = ref**2 + side_size - n
        print("----right",d)
        y = (side_size//2) - d
    
    elif n <= ref**2 + 2*side_size:
        # top side
        y = side_size//2
        d = ref**2 + 2*side_size - n
        print("----top",d)
        x = (side_size//2) - d
    
    elif n <= ref**2 + 3*side_size:
        # left side
        x = side_size//2
        d = ref**2 + 3*side_size - n
        print("----left",d)
        y = (side_size//2) - d
        
    else:
        # bottom side
        y = side_size//2
        d = ref**2 + 4*side_size - n
        print("----bottom",d)
        x = (side_size//2) - d
        
    print("-----",x,y)
    return x + y

In [248]:
get_md_spiral_mem(347991)

------ 589 1070 590
----top 110
----- 185 295


480

### part 2 (incomplete)

## Day 4: High-Entropy Passphrases

In [249]:
! cat day4_input.txt

pphsv ojtou brvhsj cer ntfhlra udeh ccgtyzc zoyzmh jum lugbnk
vxjnf fzqitnj uyfck blnl impo kxoow nngd worcm bdesehw
caibh nfuk kfnu llfdbz uxjty yxjut jcea
qiho qif eupwww avyglnj nxzotsu hio lws
xjty usocjsh pivk qnknunc yjcgh bwya djw zpyr
ycfmfe mgq sjiomg nfzjul bjwkmgu yvsnvgj dcjupu wzz blmn
rdowgbt vpwfdoi blzl laghnk gsa vhnpo cztxzlb rtz hvwonhb eciju pfjtbo
bqs bqs dbutvgf mmzb izpyud rap izpyud xlzeb mnj hjncs
xpu vwp nujcos piu irindir tpmfd umtvlm gznu
sfpuxar qcnbte omouazv cnh uaxspfr sepolf rusafpx
xbmaf iceyqqq sabpt gliexel muubepe qqiyqce fmrcc eazk obkeonl fmccr kgk
apg gbycwe gap pag
gagv saqbk lwtllc wnhzz khxsjc
lgc alen rlmsp anel gcbvg
bujlaz rks rlqf deknmee yrp
scqvl weusbc bgvaz vgg cjwsfno vqy zbq aqy tvf bgzav
hbki vei fxdwljs myjuba elbsib pvy xxjxgi dtgv
linzaeu qbwdke fdg pykw
qvtdd aco aav bpu mvkcuc kjfj japgfki jfdl gem hog bdzsiea
wpbigkb lzhwba jssjkn qvb kmwu qddv
iny osyvqnt tumunzb torq bdeneg wywank poza ipp iggorw
tuko mhdbsf vmjdop jomaqpj r

In [15]:
input_data = []
with open("day4_input.txt") as f:
    for line in f.read().split("\n"):
        input_data += [list(line.split())] if line else []

### part 1

In [266]:
def get_valid1(list_of_strings):
    valids = 0
    for l in list_of_strings:
        valids +=  1 if len(set(l)) == len(l) else 0
    return valids

In [267]:
assert get_valid([['aa','bb','cc','dd','ee']]) == 1
assert get_valid([['aa','bb','cc','dd','aaa']]) == 1
assert get_valid([['aa','bb','cc','dd','aa']]) == 0


In [268]:
get_valid(input_data)

466

### part 2

In [41]:
import collections
def get_valid2(list_of_strings):
    valids = 0
    for l in list_of_strings:
        counts = [collections.Counter(s) for s in l]
        valids += 1 if all([c1 != c2 for i,c1 in enumerate(counts) for c2 in counts[i+1:]]) else 0
    return valids

In [46]:
assert get_valid2([["abcde","fghij"]]) == 1
assert get_valid2([["abcde","xyz","ecdab"]]) == 0
assert get_valid2([["a","ab","abc","abd","abf","abj"]]) == 1
assert get_valid2([["iiii","oiii","ooii","oooi","oooo"]]) == 1
assert get_valid2([["oiii","ioii","iioi","iiio"]]) == 0

In [42]:
get_valid2(input_data)

251

## Day 5: A Maze of Twisty Trampolines, All Alike

In [47]:
! cat day5_input.txt

0
0
0
2
2
-1
-3
-3
0
-6
-9
0
-1
-12
-9
-2
0
-14
-6
-2
-10
-12
-10
-13
-2
1
-6
-14
-2
-11
-7
-9
-15
-22
-25
-26
-19
-28
1
-2
-38
-39
-30
-18
0
-26
-1
-24
-1
-30
-44
-35
-9
-32
-5
-34
-4
-15
-21
-30
-10
-32
-19
-40
-12
-49
-58
-2
-14
-51
-37
-9
-4
-48
-64
-37
-55
-40
-37
2
-22
-68
-57
-57
-83
-65
-38
-22
-20
-78
-27
-40
-4
-83
-33
-47
-74
-41
-74
-68
-18
-8
-27
-23
-53
-70
-43
-99
-48
-90
1
-74
-9
-80
-96
-33
-7
-53
-98
-54
-47
-84
-81
-79
-86
-14
-115
-121
-30
-28
-13
-113
-41
-20
-34
-19
-71
-39
-17
-91
-115
-108
-74
-134
-12
-91
-27
-59
-27
-132
-34
-8
-52
-93
-17
-151
-93
-102
-62
-62
-120
-25
-75
-35
-162
-61
-107
-83
-106
-23
-168
-42
-13
-74
-52
-169
-123
-95
-174
-56
-43
-84
-21
-5
-120
-130
-55
-3
-93
-158
-61
-4
-74
-65
-157
-112
-147
-24
-23
-155
-82
-73
-25
-82
-42
-200
-120
-57
-96
-38
-121
-25
-211
-117
-42
-181
-56
-204
-193
-88
-143
-167
-42
-178
-204
-57
-120
-33
-164
-122
-219
-136
-174
0
-36
-64
-77
-34
-158
-163
-194
-212
-101
-48
-216
-141
-142
-189
-101
-144
-158
-1

In [20]:
input_data = []
with open("day5_input.txt") as f:
    input_data = list(map(int,f.read().strip().split("\n")))

### part 1

In [21]:
def when_out1(list_of_nos):
    l = list_of_nos[:]
    current_index = 0
    steps = 0
    while current_index <= len(l)-1 and current_index >= 0:
        # print("----",steps, l,current_index, current_index <= len(l)-1, current_index >= 0)
        steps += 1
        forward_index_by = l[current_index]
        l[current_index] += 1
        current_index += forward_index_by
    return  steps

In [22]:
assert(when_out1([0,3,0,1,-3])) == 5

In [23]:
when_out1(input_data)

394829

### part 2

In [24]:
def when_out2(list_of_nos):
    l = list_of_nos[:]
    current_index = 0
    steps = 0
    while current_index <= len(l)-1 and current_index >= 0:
        # print("----",steps, l,current_index, current_index <= len(l)-1, current_index >= 0)
        steps += 1
        forward_index_by = l[current_index]
        if forward_index_by >= 3:
            l[current_index] -= 1
        else:
            l[current_index] += 1
        current_index += forward_index_by
    return  steps

In [29]:
assert(when_out2([0,3,0,1,-3])) == 10

In [28]:
when_out2(input_data)

31150702

## Day 6: Memory Reallocation

In [155]:
! cat day6_input.txt

2	8	8	5	4	2	3	1	5	5	1	2	15	13	5	14


In [156]:
input_data = []
with open("day6_input.txt") as f:
    input_data = list(map(int,f.read().strip().split()))

### part 1

In [177]:
import numpy as np
def cycling_reallocated_buffers1(buffers):
    l = np.array(buffers)
    n = len(l)
    diviup_by = n-1
    visited_buff_configs = set()
    buffer_config = "-".join(list(map(str,l)))
    step = 1
    while buffer_config not in visited_buff_configs:
        visited_buff_configs.add(buffer_config)
        max_index = np.argmax(l)
        divided = l[max_index]//diviup_by
        remainder = l[max_index]%diviup_by
        if divided == 0:
            for i in range(1,l[max_index]+1):
                l[(max_index+i)%n] += 1
            l[max_index] = 0
        else:
            l += divided
            l[max_index] = 0
            l[max_index] += remainder
            
        buffer_config = "-".join(list(map(str,l)))
        step += 1
    
    return step - 1

In [178]:
assert cycling_reallocated_buffers1([0,2,7,0]) == 5

In [179]:
cycling_reallocated_buffers1(input_data)

3156

### part 2

In [180]:
import numpy as np
def cycling_reallocated_buffers2(buffers):
    l = np.array(buffers)
    n = len(l)
    diviup_by = n-1
    visited_buff_configs = {}
    buffer_config = "-".join(list(map(str,l)))
    step = 1
    while buffer_config not in visited_buff_configs:
        visited_buff_configs[buffer_config] = step
        max_index = np.argmax(l)
        divided = l[max_index]//diviup_by
        remainder = l[max_index]%diviup_by
        if divided == 0:
            for i in range(1,l[max_index]+1):
                l[(max_index+i)%n] += 1
            l[max_index] = 0
        else:
            l += divided
            l[max_index] = 0
            l[max_index] += remainder
            
        buffer_config = "-".join(list(map(str,l)))
        step += 1
    
    return step - visited_buff_configs[buffer_config]

In [182]:
assert cycling_reallocated_buffers2([0,2,7,0]) == 4

In [183]:
cycling_reallocated_buffers2(input_data)

1610

## Day 7: Recursive Circus

In [184]:
! cat day7_input.txt

fjkfpm (69) -> kohxzh, liwvq, eqkio, xvoyybs
dsiixv (52)
fhimhm (66)
mdlubuq (73)
ulobwyb (41)
cbgnzhz (70)
hrheyzf (946) -> fixqj, msyvs, pdwjcd, tlgdija
yrnjhqc (31)
xlglga (714) -> hohft, funabvw, zhxnpoh, unpdcwm
hcvwov (240)
qnhvjec (35016) -> pimzjjp, ghatw, tbakk, olgmto, qeaqiuq, wnmnz, aabbf
efgdfa (405) -> uyedi, rxkmf, syhipz, zozcoqd, uwprsvh, dqhoh, rcbhgw
vofrtf (55)
gipbso (52)
cviwy (426) -> ildgly, syojg, tgoztyw, doavm
okzkfw (159) -> ndteuz, bkmepbs
ylfvgv (5)
ydxuhqe (72) -> bfftnj, uwrll, njlaepf, kbfpx
hwfcy (54)
iqtdzrh (65) -> qoppte, jcsqqxv
ddumyx (220) -> yjowio, gefjk
edagh (12)
kefudoj (82)
indjoox (36)
cliojs (35)
lrvmyj (79) -> fadkfic, pcikcq, fgeup
mvyzuw (73)
nholw (52)
sjbqzk (212) -> sijon, bcedyx
majskr (49)
qjutng (15)
iavwfbb (19)
ywonug (50) -> dgjyf, kobwktw
taxky (77)
lufkiy (44)
lcwqdxx (34)
felksho (204) -> vkile, ihadu, qfczh, wvgzlw, axawjtm, eutun, vkzmf
rvtus (94)
zjxjhwc (53)
bnmsz (95)
cvcyykp (31)
bfxvxpl (35)
niavkxg (16)
zwcsjgs (7)


In [194]:
input_data = {}
import re
with open("day7_input.txt") as f:
    for line in f.read().strip().split("\n"):
        m = re.match(r"([a-z]+)\s+\(\d+\)\s*[->\s]*(.*)",line)
        input_data[m.group(1)] = m.group(2).split(", ") if m.group(2) else []
        
input_data

{'fjkfpm': ['kohxzh', 'liwvq', 'eqkio', 'xvoyybs'],
 'dsiixv': [],
 'fhimhm': [],
 'mdlubuq': [],
 'ulobwyb': [],
 'cbgnzhz': [],
 'hrheyzf': ['fixqj', 'msyvs', 'pdwjcd', 'tlgdija'],
 'yrnjhqc': [],
 'xlglga': ['hohft', 'funabvw', 'zhxnpoh', 'unpdcwm'],
 'hcvwov': [],
 'qnhvjec': ['pimzjjp',
  'ghatw',
  'tbakk',
  'olgmto',
  'qeaqiuq',
  'wnmnz',
  'aabbf'],
 'efgdfa': ['uyedi',
  'rxkmf',
  'syhipz',
  'zozcoqd',
  'uwprsvh',
  'dqhoh',
  'rcbhgw'],
 'vofrtf': [],
 'gipbso': [],
 'cviwy': ['ildgly', 'syojg', 'tgoztyw', 'doavm'],
 'okzkfw': ['ndteuz', 'bkmepbs'],
 'ylfvgv': [],
 'ydxuhqe': ['bfftnj', 'uwrll', 'njlaepf', 'kbfpx'],
 'hwfcy': [],
 'iqtdzrh': ['qoppte', 'jcsqqxv'],
 'ddumyx': ['yjowio', 'gefjk'],
 'edagh': [],
 'kefudoj': [],
 'indjoox': [],
 'cliojs': [],
 'lrvmyj': ['fadkfic', 'pcikcq', 'fgeup'],
 'mvyzuw': [],
 'nholw': [],
 'sjbqzk': ['sijon', 'bcedyx'],
 'majskr': [],
 'qjutng': [],
 'iavwfbb': [],
 'ywonug': ['dgjyf', 'kobwktw'],
 'taxky': [],
 'lufkiy': [],
 'lcwq

### part 1:

In [ ]:
def find_source(inpu)

### part 2: